TensorFlow提供了一种统一的数据存储格式，这就是TFRecord。
## TFRecord格式介绍
TFRecord文件中的数据是通过tf.train.Example Protocol Buffer的格式存储。以下为tf.train.Example的代码格式：
```python
message Example {
    Features features = 1;
}
message Features {
    map<string, Feature> feature = 1;
}
message Feature {
    oneof kind {
        BytesList bytes_list = 1;
        FloatList float_list = 2;
        Int64List int64_list = 3;
    }
}
```
Example中包含了一个从属性名称到取值的字典，其中属性名称为字符串，取值可以为字符串、实数列表或整数列表。比如将一张解码前的图片存为一个字符串，图像所对应的类别编号存为整数列表。
## TFRecord使用样例
### 写入TFRecord文件
以下程序给出将MNIST输入数据转化为TFRecord格式。

In [5]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

# 生成整数型的属性
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

# 生成字符串性的属性
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

mnist = input_data.read_data_sets(
    '../Data', dtype=tf.uint8, one_hot=True)
images = mnist.train.images
labels = mnist.train.labels

pixels = images.shape[1]
num_examples = mnist.train.num_examples
# 输出TFRecord文件的目录
filename = './out/output.tfrecords'
writer = tf.python_io.TFRecordWriter(filename)

for index in range(num_examples):
    # 将图像矩阵转换为字符串
    image_raw = images[index].tostring()
    # 将一个样本转换为Example Protocol Buffer，并将所有信息写入这个数据结构
    example = tf.train.Example(
        features = tf.train.Features(feature={
            'pixels': _int64_feature(pixels),
            'labels': _int64_feature(np.argmax(labels[index])),
            'image_raw': _bytes_feature(image_raw)
        })
    )
    writer.write(example.SerializeToString())
writer.close()

Extracting ../Data/train-images-idx3-ubyte.gz
Extracting ../Data/train-labels-idx1-ubyte.gz
Extracting ../Data/t10k-images-idx3-ubyte.gz
Extracting ../Data/t10k-labels-idx1-ubyte.gz


以上程序是将所有样本数据存储到一个TFRecord文件中，当数据量比较大的时候，可以将数据写入不同的TFRecord文件。
### 读取TFRecord文件
TensorFlow可以从文件列表中读取数据。

In [10]:
import tensorflow as tf

# 创建reader来读取TFRecord
reader = tf.TFRecordReader()
# 创建队列来维护文件列表
filename_queue = tf.train.string_input_producer(['./out/output.tfrecords'])

_, serialized_example = reader.read(filename_queue)
# 从文件中读取一个样本。可以使用parse_examle函数读取多个样例
features = tf.parse_single_example(
    serialized_example, 
    features={
        # TF提供两种不同的属性解析函数。tf.FixedLenFeature方法解析结果
        # 为一个tensor, tf.VarLenFeature方法解析结果为SparseTensor，
        # 用于处理稀疏数据
        'pixels': tf.FixedLenFeature([], tf.int64),
        'labels': tf.FixedLenFeature([], tf.int64),
        'image_raw': tf.FixedLenFeature([], tf.string)
    })

# tf.decode_raw可以将字符创转换成图像对应的像素数组
image = tf.decode_raw(features['image_raw'], tf.uint8)
label = tf.cast(features['labels'], tf.int64)
pixels = tf.cast(features['pixels'], tf.int64)

sess = tf.Session()
# 启用多线程处理输入数据
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)
# 每次运行可以读取TFRecord文件中的一个特例
for i in range(10):
    print(sess.run([image, label, pixels]))

[array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  